<a href="https://colab.research.google.com/github/ClovisMonteiro/Redes_neurais/blob/master/Arquitetura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definindo uma arquitetura no PyTorch

### Carregando dados

Exemplo de dados não linerares separáveis( Distribuição em lua ):
- https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html


## nn.Sequential

O módulo nn.Sequantial é um container onde se pode colocar múltiplos módulos. Ao realizar um foward em um objeto sequential apricará sequencialmente os módulos nele contidaspara gerar uma saída.
Exemplo abaixo desse módulo contendo 2 camadas Linear intercaladas por uma função de ativação ReLU.